In [1]:
import os
import pandas as pd
import glob
from pathlib import PurePosixPath
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data_files = [os.path.join(dp, f) for dp, dn, filenames in os.walk(os.getcwd() + '\\data') for f in filenames]
data_titles = [text[text.index('data')+5:].replace('\\', '/') for text in data_files]

In [4]:
tfidf_vectorizer = TfidfVectorizer(input='filename', decode_error='ignore')
tfidf_vector = tfidf_vectorizer.fit_transform(data_files)

In [11]:
tfidf_df = pd.DataFrame(tfidf_vector.astype('float32').toarray().astype('float16'), index=data_titles, columns=tfidf_vectorizer.get_feature_names_out())

In [ ]:
tfidf_df.T['GP/part6/msg9150.eml'].sort_values(ascending=False).head(25)

In [ ]:
tfidf_df.T['lokay-m/articles/1'].sort_values(ascending=False).head(25)

In [ ]:
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [ ]:
tfidf_df['Spam'] = [1 if re.match('GP\/part[0-9]*\/.*\.eml', filename) else 0 for filename in data_titles]

In [ ]:
X_train = tfidf_df.filter(regex='(GP\/part[0-9]\/.*\.eml)|(kitchen-l\/.*)', axis=0)
y_train = X_train['Spam']
X_train = X_train.drop('Spam', axis = 1)
X_test = tfidf_df.filter(regex='(GP\/part[0-9][0-9]\/.*\.eml)|(lokay-m\/.*)', axis=0)
y_test = X_test['Spam']
X_test = X_test.drop('Spam', axis=1)

In [ ]:
rfc_model = RandomForestClassifier(n_estimators=100)

In [ ]:
rfc_model.fit(X_train, y_train)
y_fit = rfc_model.predict(X_test)
cfm = confusion_matrix(y_test, y_fit)